In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. Installer AutoKeras
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 61.4 MB/s eta 0:00:00


## Étape 0. Importer les librairies *(En continu...)*
Importez les librairies nécessaires pour l'exécution des étapes suivantes

In [3]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import autokeras as ak
import os

## Étape 1. Analyse et traitement des  données


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/ML/titanic/train.csv')
from sklearn.model_selection import train_test_split

#discretisation de la colonne 'Fare'
from sklearn.preprocessing import KBinsDiscretizer
kbins = KBinsDiscretizer(encode='ordinal')
kbins.fit(df[['Fare']])
df[['Fare']] = kbins.transform(df[['Fare']])

#Titanic enlever les NAN
mod = df['Embarked'].mode()
df['Embarked'].fillna(value=mod, inplace=True)
df['Embarked'].fillna(value=mod, inplace=True)
moy = df['Age'].mean()
df['Age'].fillna(value=moy, inplace=True)
df['Age'].fillna(value=moy, inplace=True)

df.dropna(inplace=True)

# Transformer nos données discrètes en données continues - OHE
df = pd.get_dummies(df,columns=['Sex','Embarked'])
df = df.drop(['Name','Ticket','Cabin', 'PassengerId'], axis=1)

#separer nos x de nos y
y = df['Survived'].to_numpy()
X = df.drop(['Survived'], axis=1).to_numpy()
df.head()

Survived  Pclass   Age  SibSp  Parch  Fare  Sex_female  Sex_male  \
1          1       1  38.0      1      0   4.0           1         0   
3          1       1  35.0      1      0   4.0           1         0   
6          0       1  54.0      0      0   4.0           0         1   
10         1       3   4.0      1      1   2.0           1         0   
11         1       1  58.0      0      0   3.0           1         0   

    Embarked_C  Embarked_Q  Embarked_S  
1            1           0           0  
3            0           0           1  
6            0           0           1  
10           0           0           1  
11           0           0           1

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [6]:
X_train.shape

(161, 10)

## Étape 3. Création d'un modèle autokeras Prédire la survie du passager


In [7]:
model = ak.StructuredDataClassifier(max_trials=10, overwrite=True , metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.7)], objective="val_binary_accuracy", seed=42)
model.fit(
  x = np.asarray(X_train).astype(np.float32),
  y = np.asarray(y_train).astype(np.float32),
  validation_data = (X_test, y_test)
)

Trial 10 Complete [00h 00m 06s]
val_binary_accuracy: 0.7560975551605225

Best val_binary_accuracy So Far: 0.8048780560493469
Total elapsed time: 00h 01m 07s


In [55]:
# Obtenir le meilleur modele Keras, a partir du modele AutoKeras
model_keras = model.export_model()

# Sauvegarder le modele Keras, sous son extension .tf
tf.keras.models.save_model( model_keras , "/content/drive/MyDrive/Dataset/DML2/best_dnn.tf", save_format='tf' )

### 3.3 Évaluation et sauvegarde du modèle


In [56]:

# Lire le modele
model_charge = tf.keras.models.load_model("/content/drive/MyDrive/Dataset/DML2/best_dnn.tf")
model_charge.summary()

# Utiliser le modele Keras pour faire une/des prediction(s)
prediction = model_charge.predict(np.asarray(X_train).astype(np.float32))
print(prediction)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 9)                0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                320       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  